In [ ]:
!pip install scipy

In [2]:
# Simulate swapping with and without price scale
# Example setup
# swap WBTC to USDC
# x = USDC
# y = WBTC
# market price of WBTC = 60,000

from scipy.optimize import fsolve

# Functions to solve for x, y and D
def f_curve_v2(x, y, A, D, g, p):
    # Transformed balances
    x = x * p[0]
    y = y * p[1]
    # Curve v2
    k0 = x*y/(D/2)**2
    k = A*k0*(g/(g+1-k0))**2
    return x*y + k*D*(x+y) - (D/2)**2 - k*D**2

def f_y(y, x, A, D, g, p):
    return f_curve_v2(x, y, A, D, g, p)

def f_D(D, x, y, A, g, p):
    return f_curve_v2(x, y, A, D, g, p)

# Compare dx with price scale = [1, 1] and [1, 6e4]
A = 10
g = 0.002
x0 = 6e6
y0 = 100
y1 = y0 + 1

## Calc dx with actual balances
print("--- dx with price scale = [1, 1] ---")

p = [1, 1]
D = fsolve(f_D, x0/2, args=(x0, y0, A, g, p))[0]

print("D", D)
print("D/2", D / 2)

x1 = fsolve(f_curve_v2, D/2, args=(y1, A, D, g, p))[0]
print("x1", x1)
print("dx", x0 - x1)

## Calc dx with transformed balances
print("--- dx with price scale = [1, 6e4] ---")

p = [1, 6e4]
D = fsolve(f_D, x0/2, args=(x0, y0, A, g, p))[0]

print("D", D)
print("D/2", D / 2)

x1 = fsolve(f_curve_v2, D/2, args=(y1, A, D, g, p))[0]
print("x1", x1)
print("dx", x0 - x1)

--- dx with price scale = [1, 1] ---
D 55971.23259730738
D/2 27985.61629865369
x1 5945653.999865564
dx 54346.000134436414
--- dx with price scale = [1, 6e4] ---
D 12000000.000000028
D/2 6000000.000000014
x1 5940031.185196907
dx 59968.81480309274


In [3]:
# Example of repegging the price scales
# Calculate dy with transformed balances
print("--- dy with price scale = [1, 6e4] ---")

p = [1, 6e4]
D = fsolve(f_D, x0/2, args=(x0, y0, A, g, p))[0]

print("D", D)
print("D/2", D / 2)

dx = 680000
x1 = x0 + dx
y1 = fsolve(f_y, D/2, args=(x1, A, D, g, p))[0]
print("D / 2 - x1 * p[0]", D / 2 - x1 * p[0])
print("D / 2 - y1 * p[1]", D / 2 -  y1 * p[1])
print("x1", x1)
print("y1", y1)
print("x1 * p[0]", x1 * p[0])
print("y1 * p[1]", y1 * p[1])
print("dy", y0 - y1)
print("arb profit", (y0 - y1) * 70000 - dx)

# Calculate dy with transformed balances
print("--- price scale = [1, 7e4] ---")
p = [1, 7e4]
D = fsolve(f_D, x1/2, args=(x1, y1, A, g, p))[0]

print("D", D)
print("D/2", D / 2)
print("D / 2 - x1 * p[0]", D / 2 - x1 * p[0])
print("D / 2 - y1 * p[1]", D / 2 -  y1 * p[1])
print("x1 * p[0]", x1 * p[0])
print("y1 * p[1]", y1 * p[1])

--- dy with price scale = [1, 6e4] ---
D 12000000.000000028
D/2 6000000.000000014
D / 2 - x1 * p[0] -679999.999999986
D / 2 - y1 * p[1] 645452.725933726
x1 6680000.0
y1 89.24245456777146
x1 * p[0] 6680000.0
y1 * p[1] 5354547.274066288
dy 10.75754543222854
arb profit 73028.18025599781
--- price scale = [1, 7e4] ---
D 12926234.118568232
D/2 6463117.059284116
D / 2 - x1 * p[0] -216882.94071588386
D / 2 - y1 * p[1] 216145.23954011407
x1 * p[0] 6680000.0
y1 * p[1] 6246971.819744002
